## Tabular training

In [51]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib pandas

To illustrate the tabular application, we will use the example of the Adult dataset where we have to predict if a person is earning more or less than $50k per year using some general data.

In [52]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

## Variables

In [53]:
yNames = ['Future Year Change']
catNames = []
contNames = [
    'Open',
    'High', 
    'Low', 
    'Close', 
    'Volume', 
    'Dividends', 
    'Stock Splits', 
    'EV/EBIT', 
    'Market Cap', 
    'ROIC'
]



We can download a sample of this dataset with the usual untar_data command:

In [54]:
dataPath = Path()
dataPath.ls()

(#4) [Path('data.csv'),Path('screener.ipynb'),Path('tabular.ipynb'),Path('Tutorial')]

Then we can have a look at how the data is structured:

In [55]:
df = pd.read_csv(dataPath/'data.csv')
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,EV/EBIT,Market Cap,ROIC
0,2022-01-18 00:00:00-05:00,AAPL,168.745802,169.759200,166.679656,167.063370,90956700,0.0,0.0,-0.198733,43.750052,2.512282e+12,0.018057
1,2022-01-19 00:00:00-05:00,AAPL,167.260163,168.322759,163.265599,163.550919,94815000,0.0,0.0,-0.165793,42.849538,2.459462e+12,0.018437
2,2022-01-20 00:00:00-05:00,AAPL,164.288836,166.945318,161.533959,161.858643,91420500,0.0,0.0,-0.137262,42.415676,2.434014e+12,0.018625
3,2022-01-21 00:00:00-05:00,AAPL,161.770080,163.649301,159.684253,159.792480,122848900,0.0,0.0,-0.117313,41.885958,2.402943e+12,0.018861
4,2022-01-24 00:00:00-05:00,AAPL,157.440994,159.684246,152.206728,159.015198,162294600,0.0,0.0,-0.117168,41.686680,2.391255e+12,0.018951


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in TabularDataLoaders factory methods:

In [56]:
dls = TabularDataLoaders.from_csv(dataPath/'data.csv', path=dataPath, 
    y_names=yNames,
    cat_names=catNames,
    cont_names=contNames,
    procs = [Categorify, FillMissing, Normalize])

The last part is the list of pre-processors we apply to our data:

* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s TabularPandas class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [57]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [58]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
    y_names=yNames,
    cat_names = catNames,
    cont_names = contNames,
    splits=splits)

Once we build our TabularPandas object, our data is completely preprocessed as seen below:

In [59]:
to.xs.iloc[:2]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC
18975,-0.330337,-0.324837,-0.325518,-0.321193,-0.292187,-0.051811,-0.011015,-1.101432,-0.485984,1.316726
11177,-0.388882,-0.392510,-0.397711,-0.397322,0.537752,-0.051811,-0.011015,0.195993,-0.278744,-0.537509


Now we can build our DataLoaders again:

In [60]:
dls = to.dataloaders(bs=64)

The show_batch method works like for every other application:

In [61]:
dls.show_batch()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change
0,424.579992,429.609978,423.149991,427.150001,7.055999e+05,1.674735e-10,-2.995880e-11,66.536324,6.602800e+10,0.011873,0.393094
1,141.367632,144.583910,141.357775,144.386596,6.625370e+07,1.674735e-10,-2.995880e-11,37.936233,2.171271e+12,0.020824,0.296455
2,307.510008,308.779998,303.959994,308.220002,1.261601e+06,1.674735e-10,-2.995880e-11,89.446326,1.097815e+11,0.008832,0.293946
3,407.797604,412.852146,403.216954,404.056089,1.261399e+06,1.674735e-10,-2.995880e-11,46.301162,1.131022e+11,0.017062,0.597253
4,46.820004,48.589992,46.740001,48.289993,1.938799e+06,1.674735e-10,-2.995880e-11,20.880463,9.755678e+09,0.037834,0.210603
5,42.054803,42.216496,41.465430,41.654859,2.412000e+07,1.674735e-10,-2.995880e-11,31.984528,1.952514e+11,0.024699,1.091557
6,157.071976,159.901947,156.966812,159.729861,2.981700e+06,1.674735e-10,-2.995880e-11,58.310830,7.927346e+10,0.013548,0.099703
7,45.202648,45.392759,43.609292,43.672660,7.488301e+06,1.674735e-10,-2.995880e-11,25.993894,4.388361e+10,0.030392,-0.082114
8,141.929992,151.100006,139.539995,149.949995,4.838201e+06,1.674735e-10,-2.995880e-11,68.479249,5.770346e+10,0.011536,-0.176125
9,116.000000,119.350003,114.760002,115.660007,6.880230e+07,1.674735e-10,-2.995880e-11,13.832080,1.216165e+12,0.057114,0.104444


We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.

Note: Sometimes with tabular data, your y’s may be encoded (such as 0 and 1). In such a case you should explicitly pass y_block = CategoryBlock in your constructor so fastai won’t presume you are doing regression.

In [62]:
learn = tabular_learner(dls, metrics=[rmse, mae])

And we can train that model with the fit_one_cycle method (the fine_tune method won’t be useful here since we don’t have a pretrained model).

In [63]:
learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,_rmse,mae,time
0,0.221130,0.215603,0.464331,0.335086,00:02
1,0.160258,7113.971191,84.344360,1.584508,00:02
2,0.128901,26.631933,5.160614,0.339887,00:02
3,0.129306,1.306551,1.143045,0.262222,00:02
4,0.133109,215.904602,14.693692,0.483673,00:02
5,0.118209,9.576631,3.094613,0.300949,00:02
6,0.122682,51.198318,7.155300,0.368221,00:02
7,0.125024,0.176915,0.420613,0.253905,00:02
8,0.125383,2972.336914,54.519146,1.100251,00:02
9,0.115616,16.168350,4.020989,0.312600,00:02


We can then have a look at some predictions:

In [64]:
learn.show_results()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change,Future Year Change_pred
0,0.140168,0.129623,0.122791,0.114231,-0.308828,-0.051811,-0.011015,2.284516,-0.365262,-1.001892,-0.068241,0.241369
1,-0.267875,-0.268031,-0.264229,-0.262873,-0.288051,-0.051811,-0.011015,-0.988520,-0.471758,0.895915,0.125427,0.569559
2,-0.516748,-0.517340,-0.515097,-0.514961,4.709630,-0.051811,-0.011015,1.921165,1.585181,-0.959324,2.216739,1.503917
3,-0.110478,-0.113678,-0.106352,-0.107376,-0.290349,-0.051811,-0.011015,-0.419276,-0.309220,-0.117479,0.172043,0.090592
4,-0.489998,-0.486685,-0.487522,-0.484418,0.138783,-0.051811,-0.011015,0.333975,-0.045328,-0.597250,0.430369,0.512951
5,-0.473946,-0.476061,-0.472121,-0.474494,-0.263844,-0.051811,-0.011015,-0.821086,-0.404922,0.465381,0.173320,0.135867
6,-0.178918,-0.184628,-0.175839,-0.179878,-0.291464,-0.051811,-0.011015,1.659898,-0.256205,-0.923068,0.125758,0.057031
7,-0.468522,-0.468184,-0.467441,-0.470368,-0.056095,-0.051811,-0.011015,0.508093,0.003360,-0.662102,1.124512,0.649612
8,0.155077,0.144693,0.154432,0.153360,-0.312311,-0.051811,-0.011015,0.485043,-0.412351,-0.654103,0.249378,0.401659



To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [72]:
predictionTarget = 'NATH'

from ipynb.fs.fullscreener import getTickerData

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)

Static Data for NATH:
  Total Debt: 65578000
  Cash: 31207000
  Shares Outstanding: 4084620
Approximated EBIT for NATH: 21563548.8


Then Learner.get_preds will give you the predictions:

In [73]:
learn.get_preds(dl=dl)

(tensor([[0.1325]]), None)

Note:
Since machine learning models can’t magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training